# Importación de librerias

In [1]:
import numbers
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from scipy.spatial import distance
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn import svm

# Definición del método para el cálculo del AUC

In [2]:
"""
Método para el cálculo del Area Under the Curve
Parámetros
----------
user_scores : array con los scores o distancias del usuario legítimo
imposter_scores : array con los scores o distancias de usuarios ilegítimos

Retorno
-------
AUC: area bajo la curva ROC
"""
def evaluateAUC(user_scores, imposter_scores):
    #Se etiquetan los usuarios legítimos con 0 e impostores con 1
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    auc_score = roc_auc_score(labels, user_scores + imposter_scores)
    return auc_score

# Definición del método para el cálculo del Equal Error Rate
Esta función toma como entrada los puntajes de los usuarios legítimos e ilegítimos, forma la curva de ROC y calcula de manera geométrica el punto donde los falsos negativos y los falsos positivos son iguales.

>![Fuente sobre el cálculo del ERR](./img/Explanation%20ERR%20text.png)
>![Fuente sobre el cálculo del ERR](./img/Explanation%20ERR.png)
>
>**Fuente:** Bleha, S., Slivinsky, C., & Hussien, B. (1990). Computer-Access Security Systems Using Keystroke Dynamics. IEEE Transactions on Pattern Analysis and Machine Intelligence, 12(12), 1217–1222. https://doi.org/10.1109/34.62613

Conforme el threshold varia de manera creciente, los dos tipos de errores crecen de manera inversa. Es por ello que un threshold  "optimo" se puede determinar cuando el punto A y B coinciden en la grafica, los cuales son el fpr, tpr.

[Enlace a la fuente de la fórmula del ERR](https://stackoverflow.com/questions/28339746/equal-error-rate-in-python)

In [3]:
"""
Método para el cálculo del ERR
Parámetros
----------
user_scores : array con los scores o distancias del usuario legítimo
imposter_scores : array con los scores o distancias de usuarios ilegítimos

Retorno
-------
Equal Error Rate: punto donde los missrates y los far
"""
#Primera forma de hallar el ERR
def evaluateEER(user_scores, imposter_scores):
    #Se etiquetan los usuarios legítimos con 0 e impostores con 1
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    
    #Se utiliza el metodo de roc_curve para hallar los fpr, tpr y umbrales
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    
    #Variable con los False Negative Rate (FNR) - miss
    missrates = 1 - tpr
    
    #Variable con los False Positive Rate (FPR) - false alarm
    farates = fpr
    
    #Se hallan las distancias entre los FNR y FPR dado cierto umbral
    dists = missrates - farates
    
    #Listas que separan las distancias con los valores 
    #que estan más cercano al cero tanto superior como inferior
    tempList1 = dists[dists >= 0]
    tempList2 = dists[dists < 0]
    
    #Se busca el punto en la curva ROC donde se interceptan geométricamente el FNR y FPR
    #El primero que sea el cercano superior al false alarm (>=)
    #y aquel que este pegado a este en la curva pero siendo el cercano inferior (<)
    # argmin te arroja el indice el item con los menores valores
    # argmax te arroja el indice el item con los mayores valores
    #idx es una variable que almacena este índice
    
    #Indice del menor elemento del tempList1 (Lo más pegado al cero superiormente)
    idx1 = np.argmin(tempList1)
    #Sacar el indice del valor de idx1 (en tempList1), pero en la lista "dists"
    idx1, = np.where(dists == tempList1[idx1])
    
    #Indice del mayor elemento del tempList2 (Lo más pegado al cero inferiormente)
    idx2 = np.argmax(tempList2)
    #Sacar el indice del valor de idx2 (en tempList2), pero en la lista "dists"
    idx2, = np.where(dists == tempList2[idx2])
    
    #Se determina es valor de los dos puntos y ponerlo en la variable x e y
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]

    #encuentrar el punto en la línea entre x e y en donde 
    #los primeros y segundos elementos del vector sean iguales.
    #Específicamente, la línea que pasa a través de x e y 
    #se define como x + a * (y-x) para todo "a"
    #Si usamos esta formula y lo igualamos, ya q x e y 
    #deben de coincidir en ese punto de la recta
    #  -> x[1] + a*(y[1]-x[1]) = x[2] + a*(y[2]-x[2])
    #lo factorizamos para determinar a
    #que seria la pendiente de la recta que construiremos
    #  -> a = (x[1] - x[2]) / (y[2]-x[2]-y[1]+x[1]) 
    
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    
    return eer


#Segunda forma de hallar el EER.
def evaluate_EER_Thresh(user_scores, imposter_scores):
    #Se etiquetan los usuarios legítimos con 0 e impostores con 1
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    
    #Se utiliza el metodo de roc_curve para hallar los fpr, tpr y umbrales
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    
    #Se calcula el EER cuando el punto del fpr y del fpr se encuentran
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

# Lectura de archivo y eliminación de registros no válidos
Se eliminan los registros que NO hayan escrito la palabra greyc laboratory

In [4]:
#Se define la ruta donde se encuentra el archivo y se establece la conexión
path = "./data/grey/keystroke.db"
conn = sqlite3.connect(path)

#Se hace la lectura y se almacena los datos extraídos en la variable df ("dataframe")
df = pd.read_sql_query('select * from keystroke_datas', conn, parse_dates=['date'])

#Se eliminan los registros de los usuarios que no hallan escrito la palabra 'greyc laboratory'
df.drop(df[df['password'] != 'greyc laboratory'].index, inplace = True)

#Se hace un cierre de la conexión con la base de datos SQLite
conn.close()

# Previsualización del dataset original
 El siguiente dataset contiene las siguientes columnas:
  - ppTime: vector de tiempo entre dos teclas presionadas (press - press)
  - rrTime: vector de tiempo entre dos teclas soltadas (release - release)
  - prTime: vector de tiempo entre una tecla presionada y luego soltada (press - release)
  - rpTime: vector de tiempo entre una tecla soltada y luego presionada (release - press)
  - vector: este vector concatena todos los vectores anteriores en uno solo
  - password: palabra escrita  por el usuario
  - user_id: id del usuario
  - time_to_type: tiempo que tardo en escribir la palabra
  - rawPress: data cruda extraida de las teclas presionadas
  - rawRelease: data cruda extraida de las teclas soltadas  

In [5]:
df.head(5)

,id,ppTime,rrTime,prTime,rpTime,vector,password,user_id,date,time_to_type,rawPress,rawRelease
5,6,2203168 600864 1101584 1602304 801152 2303312...,2203168 901296 500720 2503600 600864 1902736 ...,3204608 1902736 1802592 3204608 2203168 33047...,1201728 -400576 -200288 901296 -801152 901296...,2203168 600864 1101584 1602304 801152 2303312...,greyc laboratory,1,2009-03-18 08:49:09,30644064,71 633729665463844160\n82 633729665466047328\n...,71 633729665464845600\n82 633729665467048768\n...
6,7,2103024 500720 2703888 1602304 2103024 340489...,1902736 701008 2203168 1802592 2203168 310446...,3104464 1702448 3404896 2503600 3104464 41059...,901296 -500720 1502160 901296 1201728 2403456...,2103024 500720 2703888 1602304 2103024 340489...,greyc laboratory,1,2009-03-18 08:49:18,33748528,71 633729665547564544\n82 633729665549667568\n...,71 633729665548766272\n82 633729665550669008\n...
7,8,2203168 701008 1402016 1301872 2303312 330475...,2503600 801152 901296 1602304 2203168 3104464...,3304752 1902736 2103024 2303312 3204608 40057...,1402016 -400576 200288 600864 1301872 2403456...,2203168 701008 1402016 1301872 2303312 330475...,greyc laboratory,1,2009-03-18 08:49:28,29442336,71 633729665651914592\n82 633729665654117760\n...,71 633729665652715744\n82 633729665655219344\n...
8,9,2103024 701008 1902736 1702448 1602304 260374...,2303312 701008 1402016 2002880 1602304 240345...,3304752 1902736 2603744 2703888 2603744 34048...,1101584 -500720 701008 1001440 600864 1602304...,2103024 701008 1902736 1702448 1602304 260374...,greyc laboratory,1,2009-03-18 08:49:48,31545360,71 633729665853404320\n82 633729665855507344\n...,71 633729665854405760\n82 633729665856709072\n...
9,10,2002880 600864 1201728 1602304 1502160 300432...,2002880 901296 701008 1802592 1502160 2703888...,2904176 1802592 1902736 2503600 2403456 36051...,1101584 -300432 0 901296 600864 2103024 50072...,2002880 600864 1201728 1602304 1502160 300432...,greyc laboratory,1,2009-03-18 08:49:58,29642624,71 633729665954750048\n82 633729665956752928\n...,71 633729665955651344\n82 633729665957654224\n...


# Preprocesamiento de vectores de tiempo

In [6]:
#Variable temporal para almacenar los items de la columna vector
#ya que no se puede trabajar directamente con ese formato
tempData = []

#Variable con la cantidad de registros totales de la bd
n_data_rows = df.shape[0]

#Variable que tendrá la lista de columnas del dataframe
columns = ["user_id"]   

#Como existen en total 60 features de tiempo por usuario, se generará 
#los nombres de las columnas siguiento el siguiente formato 
#=> 'ft_'   +  posición del feature en el vector
for i in range(60):
    columns.append("ft_" + str(i+1))

#Por cada registro que existe en la bd se aplica lo siguente
for i in range(n_data_rows):
    
    #Se extrae el usuario de ese registro
    user_id = [df.iloc[i]["user_id"]]
    #Se extrae el tiempo de tecleo
    time_to_type = [df.iloc[i]["time_to_type"]]
    
    #Se crea el vector de tiempo
    vector = df.iloc[i]["vector"].split()      
 
    #Se verifica que la integridad del vector este OK,
    #es decir que tenga una longitud exacta de 60 items
    if(len(vector) == 60 ):
        #Se aprega el registro a la variable temporarl tempData si cumple con la condición de integridad
        tempData.append(user_id  + list(map(int, vector)))

#Se crea el dataframe y se asigna a la variable df
df = pd.DataFrame(tempData, columns = columns)

#Se liberan recursos de la variable
tempData.clear()

## Previsualización de dataset procesado
Por cada registro, se tienen las siguientes columnas
 - Los features entre el 1 y 12 corresponden al vector de tiempo ppTime
 - Los features entre el 13 y 25 corresponden al vector de tiempo rrTime
 - Los features entre el 26 y 37 corresponden al vector de tiempo prTime
 - Los features entre el 38 y 60 corresponden al vector de tiempo rpTime

In [7]:
df.head(5)

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
0,1,2203168,600864,1101584,1602304,801152,2303312,1001440,2603744,1402016,...,3304752,2002880,3605184,2002880,2804032,2403456,2904176,2403456,2303312,2103024
1,1,2103024,500720,2703888,1602304,2103024,3404896,1402016,2603744,1402016,...,4105904,2303312,3505040,2103024,2203168,2603744,2804032,2603744,3104464,2103024
2,1,2203168,701008,1402016,1301872,2303312,3304752,901296,3004320,1502160,...,4005760,1902736,3905616,2203168,2303312,3104464,2603744,2203168,2203168,2603744
3,1,2103024,701008,1902736,1702448,1602304,2603744,1502160,3004320,1502160,...,3404896,2703888,4005760,2203168,2303312,2403456,3104464,2103024,2904176,2403456
4,1,2002880,600864,1201728,1602304,1502160,3004320,1101584,2904176,1602304,...,3605184,2203168,3905616,2303312,2603744,3104464,2603744,1902736,2503600,2203168


# Separación de la data de entrenamiento y de prueba
Se toma de forma aleatoria el 80% de los registros de cada usuario para considerarlos como data de entrenamiento, el 10%  para la data de desarrollo del umbral y el 10 % restante para la data de prueba


La función train_test_split cuando existe un grupo impar, siempre el último subgrupo recibe el elemento extra.
Se trató de redondear el 0.80 del parámetro **train_size** debido a que cuando se trabaja con esta cantidad, arrojaba una mayor cantidad de splits con subdatasets de dev y de test desiguales. Aproximandamente **73**
![Resultados usando una proporcion de 80](./img/Proportion%20split%2080.png)

Poner 0.84 en el parámetro **train_size** permite tener menores grupos de subdatasets de dev y de test desiguales. En este solo se obtienen **4** de este tipo.
> Se debe de mantener que el dataset de dev sea lo más similar al de test.

[Link de fuente](https://cs230.stanford.edu/blog/split/)

![Resultados usando una proporcion de 84](./img/Proportion%20split%2084.png)

In [8]:
#Variable que contiene la lista de todos los usuarios de la bd
subjects = df["user_id"].unique()

#Variable del dataset de train 80%
train_users = []

#Variable del dataset de dev (desarrollo) para el calculo del umbral 10%
dev_users = []

#Variable del dataset de test 10%
test_users = []

#Separar el df en 80 / 10 / 20 respectivamente y asignarlo a sus 
for subject in subjects:
    current_user_data = df.loc[df.user_id == subject, :]
            
    #imposter_data = df.loc[df.user_id != subject, :]
    
    #Caso especial de una proporcion de 60/20/20 cuando el usuario tiene solo 5 registros
    #Para no eliminar ese registro y no lanze error 
    #Donde quedaría asi 5 -> 3 / 1 / 1

    if len(current_user_data) == 5:
        train, dev = train_test_split(current_user_data, train_size = 0.6, random_state=43, shuffle=True)
        dev , test = train_test_split(dev, train_size = 0.5, random_state=43, shuffle=True)
    
    #Caso contrario se respeta la proporcion de 80/10/10 establecida antes
    else:
        train, dev = train_test_split(current_user_data, train_size = 0.80, random_state=43, shuffle=True)
        dev , test = train_test_split(dev, train_size = 0.5, random_state=43, shuffle=True)
        
    #Se agregan a los 3 datasets los splits calculados aleatoriamente
    train_users.append(train)
    dev_users.append(dev)
    test_users.append(test)

#Se convierte los arrays en dataframes manipulables
train_users = pd.concat(train_users)
dev_users = pd.concat(dev_users)
test_users = pd.concat(test_users)

### Previsualización del dataset de entrenamiento

In [9]:
train_users

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
4797,1,2103024,701008,2603744,1001440,1001440,2002880,500720,2403456,1402016,...,3004320,1702448,3605184,2203168,2703888,1602304,2203168,1702448,3404896,1602304
12,1,2403456,600864,1402016,1602304,801152,2103024,1201728,3004320,1702448,...,3204608,2303312,4306192,2503600,2603744,2703888,2703888,2103024,2603744,2203168
3959,1,1902736,701008,1001440,801152,1001440,1902736,500720,2603744,901296,...,2603744,1702448,3404896,1602304,2904176,2403456,2103024,1402016,2503600,1602304
6799,1,2002880,701008,1001440,1301872,1001440,2103024,1702448,1602304,2904176,...,2904176,2804032,2403456,4005760,3304752,2603744,2603744,1802592,2703888,1902736
2222,1,2103024,701008,901296,1201728,701008,2403456,1001440,2103024,1402016,...,3404896,1902736,3304752,2103024,3104464,2703888,2503600,1602304,3004320,1702448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7483,133,3304752,2403456,2303312,1902736,1702448,2203168,2103024,1902736,2403456,...,3404896,3004320,3104464,3304752,6409216,3104464,3304752,1902736,2403456,3705328
7480,133,1902736,2002880,2103024,2103024,1702448,3905616,1902736,2303312,2203168,...,4706768,2804032,3304752,3104464,2804032,3204608,7711088,3404896,2503600,3905616
7481,133,2203168,1802592,2203168,1602304,1402016,2603744,2203168,2103024,1602304,...,3805472,3204608,3004320,2804032,2904176,2804032,5207488,2603744,2603744,3505040
7308,133,1902736,2002880,1902736,2002880,1902736,5608064,1802592,1201728,1902736,...,6409216,2603744,2002880,3104464,3304752,3605184,5107344,2603744,3004320,3805472


### Previsualización del dataset de desarrollo

In [10]:
dev_users

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
11,1,2203168,400576,2703888,1802592,1702448,2904176,1301872,3104464,1201728,...,3605184,2303312,4105904,2203168,2403456,2503600,2603744,2203168,2703888,2103024
4789,1,1902736,701008,1502160,1101584,701008,2403456,1001440,2103024,1201728,...,3605184,2203168,3304752,2103024,3304752,2503600,2403456,1702448,3004320,2303312
2323,1,2203168,701008,1301872,1101584,801152,2303312,500720,2503600,1502160,...,3304752,1802592,3705328,2303312,3004320,2904176,2703888,2203168,2703888,2503600
8,1,2103024,600864,1201728,1702448,3104464,2403456,1402016,3505040,1502160,...,3204608,2503600,4406336,2203168,2403456,2503600,2703888,1802592,2603744,2503600
0,1,2203168,600864,1101584,1602304,801152,2303312,1001440,2603744,1402016,...,3304752,2002880,3605184,2002880,2804032,2403456,2904176,2403456,2303312,2103024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7148,132,2103024,1502160,1802592,1602304,1602304,2904176,1602304,1402016,1902736,...,3705328,2603744,2203168,2603744,3304752,2303312,1201728,3805472,3605184,2904176
7359,132,2103024,1201728,1602304,1602304,1702448,2603744,1402016,1802592,1402016,...,3404896,2403456,2403456,2203168,3404896,2403456,1402016,5507920,3605184,10314832
7140,132,2303312,1502160,2203168,1802592,2002880,3805472,1402016,1902736,1702448,...,4506480,2503600,2804032,2503600,3705328,2703888,1902736,6909936,3004320,4105904
7314,133,1902736,2103024,5608064,2804032,1301872,1902736,1402016,1702448,2103024,...,3004320,2603744,2804032,3304752,3104464,2904176,3605184,2403456,2303312,3605184


### Previsualización del dataset de prueba

In [11]:
test_users

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
6795,1,2503600,400576,1902736,1201728,701008,2203168,1301872,2303312,2804032,...,3104464,2603744,3404896,3705328,3204608,2403456,2904176,1902736,3204608,2303312
3954,1,1702448,701008,1101584,901296,701008,2002880,500720,2303312,901296,...,2804032,1602304,3404896,1702448,2804032,2203168,2203168,1602304,2603744,1502160
6802,1,1902736,701008,1502160,1101584,901296,2203168,1201728,1902736,2603744,...,3204608,2403456,2904176,3304752,2904176,2603744,2303312,1101584,2804032,1602304
405,1,1602304,901296,901296,1702448,901296,1802592,1201728,2603744,1402016,...,3004320,2303312,4206048,2403456,3104464,2603744,2503600,2403456,2503600,2303312
4791,1,2503600,801152,2103024,1201728,1001440,2103024,1201728,2403456,1802592,...,3004320,2403456,3805472,2503600,3104464,2804032,2804032,1702448,3505040,1902736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6751,132,2002880,1402016,1602304,1802592,1602304,2804032,1602304,1502160,2103024,...,3605184,2804032,2203168,2904176,3304752,2603744,1702448,5307632,3705328,3404896
6754,132,2103024,1702448,1902736,1402016,1702448,2804032,1902736,1602304,2002880,...,3805472,2804032,2603744,2904176,3605184,2403456,1902736,4306192,3505040,2403456
7477,133,2103024,2904176,3505040,3805472,1201728,2103024,4806912,1402016,2603744,...,3304752,5808352,2603744,3805472,2403456,3505040,4206048,2904176,2804032,3905616
7313,133,2203168,1802592,1702448,1201728,1902736,1902736,2103024,1402016,4005760,...,3304752,3004320,1902736,5007200,4306192,4005760,3805472,2703888,3204608,4105904


### Previsualización de los vectores con la media de los usuarios

In [12]:
train_users.head(7)

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
4797,1,2103024,701008,2603744,1001440,1001440,2002880,500720,2403456,1402016,...,3004320,1702448,3605184,2203168,2703888,1602304,2203168,1702448,3404896,1602304
12,1,2403456,600864,1402016,1602304,801152,2103024,1201728,3004320,1702448,...,3204608,2303312,4306192,2503600,2603744,2703888,2703888,2103024,2603744,2203168
3959,1,1902736,701008,1001440,801152,1001440,1902736,500720,2603744,901296,...,2603744,1702448,3404896,1602304,2904176,2403456,2103024,1402016,2503600,1602304
6799,1,2002880,701008,1001440,1301872,1001440,2103024,1702448,1602304,2904176,...,2904176,2804032,2403456,4005760,3304752,2603744,2603744,1802592,2703888,1902736
2222,1,2103024,701008,901296,1201728,701008,2403456,1001440,2103024,1402016,...,3404896,1902736,3304752,2103024,3104464,2703888,2503600,1602304,3004320,1702448
6797,1,1902736,400576,1502160,1001440,1001440,2203168,1602304,1301872,2503600,...,3204608,2503600,2002880,3505040,3805472,4606624,3905616,2002880,2703888,2403456
4798,1,1902736,500720,1602304,701008,901296,6609504,1101584,2203168,1402016,...,7410656,2103024,3505040,2203168,2904176,2804032,2904176,2002880,2603744,3304752


### Separación de los datos a usar para el usuario 1

In [13]:
#Datos del usuario 1 en el dataset de desarrollo
genuine_data = dev_users.loc[dev_users.user_id == 1, "ft_1":"ft_60"]

#Datos de los otros usuarios en el dataset de desarrollo
imposter_data = dev_users.loc[dev_users.user_id != 1, "ft_1":"ft_60"]

#Datos del usuario 1 para el entrenamiento de su modelo de SVM
train_data = train_users.loc[train_users.user_id == 1, "ft_1":"ft_60"]

### Usando One Class SVM con kernel rbf

In [14]:
#clf = svm.OneClassSVM(kernel='rbf',gamma=0.0000000000000001)
clf = svm.OneClassSVM(kernel='rbf',gamma='scale')
clf.fit(train_data)

#Variables que tendran los scores o distancias
#genuine_score = list(-clf.decision_function(genuine_data))
#imposter_score = list(-clf.decision_function(imposter_data))

genuine_score = list(clf.score_samples(genuine_data))
imposter_score = list(clf.score_samples(imposter_data))

In [15]:
clf._gamma

1.593026894632856e-14

In [16]:
#Calculo del ERR y Umbral
err,thresh = evaluate_EER_Thresh(genuine_score, imposter_score)
auc_score = evaluateAUC(genuine_score, imposter_score)
print("-------------------------------------")
print("Resultados del usuario 1")
print("ERR ==> %0.4f" % (err*100),"%")
print("Umbral de decisión ==> %0.4f" % thresh)
print("AUC ==> %0.4f" % (auc_score*100),"%")

-------------------------------------
Resultados del usuario 1
ERR ==> 90.9091 %
Umbral de decisión ==> 23.3477
AUC ==> 2.3505 %


In [17]:
user_hyperplane_distances = clf.score_samples(test_users.loc[:, "ft_1":"ft_60"])

In [18]:
#Variables de conteo de aciertos y desaciertos del usuario 1
countGood = 0
countFalse = 0

user_ids = list(test_users.loc[:, "user_id"])

for i in range(len(user_ids)):
    current_user_id = user_ids[i]
    
    current_hyperplane_distance = user_hyperplane_distances[i]
    
    if current_hyperplane_distance <= thresh:
        #Si pertenece el user_id al usuario 1 aumentar el contador de aciertos a 1, caso contrario aumentar al otro
        if current_user_id == 1:
            countGood += 1
        else:
            countFalse += 1
            
    #Si NO pertenece el user_id al usuario 1 aumentar el contador de aciertos a 1, caso contrario aumentar al otro
    else:
        if current_user_id != 1:
            countGood += 1
        else:
            countFalse += 1
            
#Imprimir aciertos
print("Total acertados "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
print("Total equivocados "+str(countFalse) + " / " + str(test_users.shape[0]) +"==> " +str(round(countFalse*100/test_users.shape[0],2)) + "%")

Total acertados 46 / 815==> 5.64%
Total equivocados 769 / 815==> 94.36%


### Usando One Class SVM con kernel lineal

In [19]:
clf = svm.OneClassSVM(kernel='linear')
clf.fit(train_data)

#genuine_score = list(-clf.decision_function(genuine_data))
#imposter_score = list(-clf.decision_function(imposter_data))

#Variables que tendran los scores o distancias
genuine_score = list(clf.score_samples(genuine_data))
imposter_score = list(clf.score_samples(imposter_data))

In [20]:
#Calculo del ERR y Umbral
err,thresh = evaluate_EER_Thresh(genuine_score, imposter_score)
auc_score = evaluateAUC(genuine_score, imposter_score)
print("-------------------------------------")
print("Resultados del usuario 1")
print("ERR ==> %0.4f" % (err*100),"%")
print("Umbral de decisión ==> %0.4f" % thresh)
print("AUC ==> %0.4f" % (auc_score*100),"%")

-------------------------------------
Resultados del usuario 1
ERR ==> 32.1477 %
Umbral de decisión ==> 7610226255022126.0000
AUC ==> 76.6820 %


In [21]:
user_hyperplane_distances = clf.score_samples(test_users.loc[:, "ft_1":"ft_60"])

In [22]:
#Variables de conteo de aciertos y desaciertos del usuario 1
countGood = 0
countFalse = 0

user_ids = list(test_users.loc[:, "user_id"])

for i in range(len(user_ids)):
    current_user_id = user_ids[i]
    
    current_hyperplane_distance = user_hyperplane_distances[i]
    
    if current_hyperplane_distance <= thresh:
        #Si pertenece el user_id al usuario 1 aumentar el contador de aciertos a 1, caso contrario aumentar al otro
        if current_user_id == 1:
            countGood += 1
        else:
            countFalse += 1
            
    #Si NO pertenece el user_id al usuario 1 aumentar el contador de aciertos a 1, caso contrario aumentar al otro
    else:
        if current_user_id != 1:
            countGood += 1
        else:
            countFalse += 1
            
#Imprimir aciertos
print("Total acertados "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
print("Total equivocados "+str(countFalse) + " / " + str(test_users.shape[0]) +"==> " +str(round(countFalse*100/test_users.shape[0],2)) + "%")

Total acertados 571 / 815==> 70.06%
Total equivocados 244 / 815==> 29.94%


### Cálculo usando todos los usuarios y distancias

In [23]:
result = []

#Variables de conteo de aciertos y desaciertos
countGood = 0
countFalse = 0

#---------------------------------------------------------------
#Determinación del umbral de los usuario

#Se hace el cálculo para cada usuario
for subject in subjects:
    
    #Considerando el sujeto actual como el genuino
    #se extraen los registros del usuario actual del dataset de desarrollo y se almacenan en la variable
    genuine_data = dev_users.loc[dev_users.user_id == subject, "ft_1":"ft_60"]

    #se extraen los registros que no pertenezcan al usuario actual del dataset de desarrollo y se almacenan en la variable
    imposter_data = dev_users.loc[dev_users.user_id != subject, "ft_1":"ft_60"]

    #Datos del usuario actual para el entrenamiento de su modelo de SVM
    train_data = train_users.loc[train_users.user_id == subject, "ft_1":"ft_60"]
    
    #Entrenamiento del modelo de SVM
    clf = svm.OneClassSVM(kernel='linear')
    clf.fit(train_data)
    
    #Variables que tendran los scores o distancias
    genuine_score = list(clf.score_samples(genuine_data))
    imposter_score = list(clf.score_samples(imposter_data))

    #Calculo del ERR y el umbral de decisión del usuario actual
    err,thresh = evaluate_EER_Thresh(genuine_score, imposter_score)
    auc_score = evaluateAUC(genuine_score, imposter_score)
    #Almacenarlo en un objeto de python el resultado
    temp_obj = {}
    temp_obj["user_id"] = subject
    temp_obj["err"] = err
    temp_obj["AUC"] = auc_score
    temp_obj["umbral"] = thresh
    temp_obj["cantRegistros"] = len(df.loc[df.user_id == subject, :])
    
    individualCountGood = 0
    individualCountFalse = 0
    
    user_hyperplane_distances = clf.score_samples(test_users.loc[:, "ft_1":"ft_60"])
    
    user_ids = list(test_users.loc[:, "user_id"])
    
    for i in range(len(user_ids)):
        current_user_id = user_ids[i]

        current_hyperplane_distance = user_hyperplane_distances[i]

        if current_hyperplane_distance <= thresh:
            #Si pertenece el user_id al usuario 1 aumentar el contador de aciertos a 1, caso contrario aumentar al otro
            if current_user_id == subject:
                countGood += 1
                individualCountGood += 1
            else:
                countFalse += 1
                individualCountFalse += 1 

        #Si NO pertenece el user_id al usuario 1 aumentar el contador de aciertos a 1, caso contrario aumentar al otro
        else:
            if current_user_id != subject:
                countGood += 1
                individualCountGood += 1
            else:
                countFalse += 1
                individualCountFalse += 1 
      
    temp_obj["Aciertos"] = str(round((individualCountGood /(individualCountGood + individualCountFalse)) *100,2)) + "%"
    temp_obj["Desaciertos"] = str(round((individualCountFalse /(individualCountGood + individualCountFalse)) *100,2)) + "%"
    
    #Agregarlo a la lista de resultados
    result.append(temp_obj)
    
#Impresión de resultados  
print("----------------------------------------------------")
print("\nERR y umbrales de cada usuario")

#Convertir en un dataframe manipulable
result = pd.DataFrame(result)
print(result)

print("\nAciertos y desaciertos")
print("Total acertados "+str(countGood) + " / " + str(test_users.shape[0] * len(subjects)) + "==> " +str(round((countGood/(test_users.shape[0] * len(subjects))) *100,2)) + "%")
print("Total equivocados "+str(countFalse) + " / " + str(test_users.shape[0] * len(subjects)) +"==> " +str(round((countFalse/(test_users.shape[0] * len(subjects))*100),2)) + "%")

----------------------------------------------------

ERR y umbrales de cada usuario
     user_id       err       AUC                 umbral  cantRegistros  \
0          1  0.321477  0.766820     7610226255022126.0            107   
1          2  0.506268  0.475216     8716249026228104.0             93   
2          3  0.734973  0.156557  1.434707864933239e+16             98   
3          4  0.449163  0.546744     6739275197161431.0             72   
4          5  0.222222  0.797787     4899269477129969.0             86   
..       ...       ...       ...                    ...            ...   
128      129  0.333333  0.769248     4522366336229634.0             58   
129      130  0.333333  0.703578     4242461949487871.0             60   
130      131  0.544595  0.281757     2780367574025533.0             24   
131      132  0.666667  0.268342     7539878034560190.0             62   
132      133  0.678378  0.231757     3168744634524407.5             24   

    Aciertos Desaciertos  